# Python 3.10: Cool New Features for You to Try

The code for this notebook is from this [RealPython article](https://realpython.com/python310-new-features/)

In [ ]:
import sys

In [ ]:
import pendulum
import requests  # Used to access random user data from `https://randomuser.me/`

## Better Error Messages

Run the examples:

- `rp310_hello.py`
- `rp310_unterminated_dict.py`
- `rp310_missing_comma.py`
- `rp310_assignment_equality.py`
- `rp310_misspell_1.py`
- `rp310_misspell_2.py`
- `rp310_misspell_3.py`

## Structural Pattern Matching

We'll demonstrate this feature with three different examples:
- Detecting and deconstructing different **structures** in your data
- Using different kinds of **patterns**
- **Matching** literal patterns

We'll also include links for more details.

### Deconstructing Data Structures

In [ ]:
a_user = {
    'name': {'first': 'Pablo', 'last': 'Galindo Salgado'},
    'title': 'Python 3.10 release manager',
}

match a_user:
    case {'name': {'first': first_name}}:
        pass

# noinspection PyUnboundLocalVariable
first_name

#### Getting Random User Data

Using `requests` to obtain different versions of the user data using the API.

In [ ]:
def get_user(version='1.3'):
    """Get random user information."""
    raw_url = f'https://randomuser.me/api/{version}/?result=1'
    # Typically, one would use the `raw_url` defined above; however, we want
    # to duplicate the random data from the
    # [RealPython article](https://realpython.com/python310-new-features/).
    # Therefore, we add a `seed` parameter to the URL with the value 310.
    #
    # Note that the JSON object returned by the API contains metadata that
    # includes **both** the version of the data as well as the seed used to
    # create the random user.
    url = f'{raw_url}&seed=310'
    response = requests.get(url)
    if response:
        return response.json()['results'][0]

In [ ]:
user_13 = get_user()
user_13

Compare the previous result with a version 1.1 random user:

In [ ]:
user_11 = get_user(version='1.1')
user_11

One of the members changed between version 1.1 and version 1.3 is `dob`. In version 1.3, the value for `dob` is a dictionary with two keys: 'data' and 'name'.


In [ ]:
user_13['dob']

In version 1.1, the value of `dob` is of type `string` containing the date of birth. As a result, the developer must **calculate** the age.

Additionally, remember that the `age` returned by the 1.3 API is accurate when the data is returned. If you store this data, this value will eventually become outdated. If this is a concern, one should calculate the current age based on the `date` field.

In [ ]:
user_11['dob']

Before Python 3.10, to calculate the age from **different** API versions, one would use an `if` statement and perform different calculations based on the API version.

In Python 3.10, we can use structural pattern matching instead!

In [ ]:
def get_age(user):
    """Get the age of a user."""
    match user:
        # Note that patterns are matched **in order**; that as the 1.3
        # version of `dob` will be tried **before** matching the 1.1 version.
        # If we reversed the order of the two `case` clauses (patterns),
        # Python would **never** match the 1.3 pattern because it would
        # always match the 1.1 version.
        #
        # The moral of the story, order candidate patterns from most
        # specific to most general.
        case {'dob': {'age': int(age)}}:
            return age
        case {'dob': dob_text}:
            print(f'{dob_text=}')
            dob = pendulum.parse(dob_text)
            now = pendulum.now(tz=dob.tz)
            return (now - dob).in_years()

In [ ]:
user_13['dob'], get_age(user_13)

In [ ]:
user_11['dob'], get_age(user_11)

### Using Different Kinds of Patterns

To gain background on structural pattern matching, consider the three PEPs related to this issue:

- PEP 634: [Specification](https://www.python.org/dev/peps/pep-0634/)
- PEP 635: [Motivation and Rationale](https://www.python.org/dev/peps/pep-0635/)
- PEP 636: [Tutorial](https://www.python.org/dev/peps/pep-0636/)

In this section, we'll learn about the different kinds of patterns that exist:

- **Mapping patterns**
  Matching mapping structures like dictionaries.
- **Sequence patterns**
  Match sequence structures like tuples and lists.
- **Capture patterns**
  Bind values to names.
- **AS patterns**
  Bind value of sub-patterns to names.
- **OR patterns**
  Match one of several different sub-patterns.
- **Wildcard patterns**
  Match anything.
- **Class patterns**
  Match class structures.
- **Value patterns**
  Match values stored in attributes.
- **Literal patterns**
  Match literal values.

In [ ]:
# A **capture pattern** is used to capture a match to a pattern **and** bind it to a name.
def sum_list(numbers):
    """Sums the numbers in a list.

    This implementation is a "port" of the classic recursive implementation
    found in languages like Scheme and Clojure.
    """
    match numbers:
        # A sequence pattern that only matches an empty sequence
        case []:
            return 0
        # Matches a sequence consisting of at least one item.
        # The expression `*rest` captures all items in the list but the first.
        case [first, *rest]:
            return first + sum_list(rest)

In [ ]:
sum_list([])

In [ ]:
sum_list([1])

In [ ]:
sum_list([1, 2, 3, 4, 5])

**Note**: Capture patterns essentially assign values to variables. A limitation is that only **undotted** names are allowed; that is, one cannot use a capture pattern to assign to a class or instance attribute directly.

Notice that `sum_list()` expects a list of **numbers** to sum. Observe what happens when you supply lists of of other values:

In [ ]:
print(sum_list("4594"))

In [ ]:
print(sum_list(4957))

This behavior is a result of the semantics of a "match". The Python interpreter attempts to match all patterns in the `match ... case ...` construct from top to bottom (in the source code). The interpreter executes that code block for the **first** matching case with the capture variables bound. If **no match** occurs, the interpreter continues executing code after the `match ... case ...` construct.

Often, though, you may want to be alerted to failed matches. To match **anything**, use the wildcard pattern, an underscore (_). A wildcard pattern **always** matches but it binds **no variables** in the `case` code block.

In [ ]:
# Illustrates the "catch all" pattern (`_`)
def sum_list_with_catch_all(numbers):
    """Sums the numbers in a list.

    This implementation is a "port" of the classic recursive implementation
    found in languages like Scheme and Clojure.
    """
    match numbers:
        # A sequence pattern that only matches an empty sequence
        case []:
            return 0
        # Matches a sequence consisting of at least one item.
        # The expression `*rest` captures all items in the list but the first.
        case [first, *rest]:
            return first + sum_list(rest)
        # Matches any other value
        case _:
            wrong_type = numbers.__class__.__name__
            raise ValueError(f'Can only sum lists, not {wrong_type}')

In [ ]:
print(sum_list_with_catch_all([]))
print(sum_list_with_catch_all([2]))
print(sum_list_with_catch_all([2, 4, 6]))

In [ ]:
def try_it(callable_to_try):
    # noinspection PyBroadException
    try:
        callable_to_try()
    except:
        exception_type, exception_value, _ = sys.exc_info()
        print(f'{exception_type.__name__}: {exception_value}')

In [ ]:
try_it(lambda: sum_list_with_catch_all('4594'))

In [ ]:
try_it(lambda: sum_list_with_catch_all(4595))

The patterns in `sum_list_with_catch_all()` are still not perfect. Consider what happens if you try to sum a list of string values.

In [ ]:
try_it(lambda: sum_list_with_catch_all(['45', '94']))

Because the base case returns the `int` value, zero (0), Python raises an exception when one tries to execute the expression `0 + '45``.

To avoid this error, one can restrict the pattern to match only integers by using a **class pattern**.

In [ ]:
# Restricting the matches to `int` values only
def sum_list_of_integers(numbers):
    """Sums the numbers in a list.

    This implementation is a "port" of the classic recursive implementation
    found in languages like Scheme and Clojure.
    """
    match numbers:
        # A sequence pattern that only matches an empty sequence
        case []:
            return 0
        # Matches a sequence consisting of at least one integer.
        # The expression `*rest` captures all items in the list but the first.
        case [int(first), *rest]:
            return first + sum_list(rest)
        # Matches any other value
        case _:
            wrong_type = numbers.__class__.__name__
            raise ValueError(f'Can only sum integer lists, not {wrong_type}')

In [ ]:
try_it(lambda: sum_list_of_integers(['45', '94']))

Adding `int()` around `first` ensures that the pattern only matches if the value bound to `first` is an `int`.

**Note**: `first` will be bound to the value **before** testing for the match (a side-effect).

The function `sum_list_of_integers()` is now a bit too restrictive; that is, it will only sum `int` value; values of type `float` (and `fraction.Fraction` and `decimal.Decimal`) will raise an exception caught in the catch all clause.

To address this last issue, we can use an **OR pattern**. An _OR pattern_ consists of two or more sub-patterns. If any of the sub-patterns match (matching left-to-right and taking the first match), the entire match succeeds with the variable bindings set in the matching sub-pattern.

In [ ]:
# Only summing values that can be summed (`int` and `float` values)
def sum_list_of_numbers(numbers):
    """Sums the numbers in a list.

    This implementation is a "port" of the classic recursive implementation
    found in languages like Scheme and Clojure.
    """
    match numbers:
        # A sequence pattern that only matches an empty sequence
        case []:
            return 0
        # Matches a sequence consisting of at least one number (either `int` or `float`).
        # The expression `*rest` captures all items in the list but the first.
        case [int(first) | float(first), *rest]:
            return first + sum_list(rest)
        # Matches any other value
        case _:
            wrong_type = numbers.__class__.__name__
            raise ValueError(f'Can only sum integer lists, not {wrong_type}')

In [ ]:
print(sum_list_of_numbers([]))
print(sum_list_of_numbers([-1]))
print(sum_list_of_numbers([1, 2.0, 3]))

In [ ]:
try_it(lambda: sum_list_of_numbers('4594'))
try_it(lambda: sum_list_of_numbers(4594))
try_it(lambda: sum_list_of_numbers(['45', '94']))